In [2]:
# load basic packages
import os
import sys
import warnings

import numpy as np
import pandas as pd
import scipy as sp
import json

import matplotlib.pyplot as plt
plt.style.use("seaborn")
import seaborn as sns
import spacy
nlp = spacy.load("en")

# save model
import pickle

# set environment
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_Colwidth", -1)
warnings.filterwarnings("ignore")

In [3]:
data = pd.read_csv("data/train.csv")

In [4]:
data.columns

Index(['channelGrouping', 'date', 'device', 'fullVisitorId', 'geoNetwork',
       'sessionId', 'socialEngagementType', 'totals', 'trafficSource',
       'visitId', 'visitNumber', 'visitStartTime'],
      dtype='object')

In [5]:
data.head(3)

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not available in demo dataset"", ""browserSize"": ""not available in demo dataset"", ""operatingSystem"": ""Windows"", ""operatingSystemVersion"": ""not available in demo dataset"", ""isMobile"": false, ""mobileDeviceBranding"": ""not available in demo dataset"", ""mobileDeviceModel"": ""not available in demo dataset"", ""mobileInputSelector"": ""not available in demo dataset"", ""mobileDeviceInfo"": ""not available in demo dataset"", ""mobileDeviceMarketingName"": ""not available in demo dataset"", ""flashVersion"": ""not available in demo dataset"", ""language"": ""not available in demo dataset"", ""screenColors"": ""not available in demo dataset"", ""screenResolution"": ""not available in demo dataset"", ""deviceCategory"": ""desktop""}",1131660440785968503,"{""continent"": ""Asia"", ""subContinent"": ""Western Asia"", ""country"": ""Turkey"", ""region"": ""Izmir"", ""metro"": ""(not set)"", ""city"": ""Izmir"", ""cityId"": ""not available in demo dataset"", ""networkDomain"": ""ttnet.com.tr"", ""latitude"": ""not available in demo dataset"", ""longitude"": ""not available in demo dataset"", ""networkLocation"": ""not available in demo dataset""}",1131660440785968503_1472830385,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"", ""bounces"": ""1"", ""newVisits"": ""1""}","{""campaign"": ""(not set)"", ""source"": ""google"", ""medium"": ""organic"", ""keyword"": ""(not provided)"", ""adwordsClickInfo"": {""criteriaParameters"": ""not available in demo dataset""}}",1472830385,1,1472830385
1,Organic Search,20160902,"{""browser"": ""Firefox"", ""browserVersion"": ""not available in demo dataset"", ""browserSize"": ""not available in demo dataset"", ""operatingSystem"": ""Macintosh"", ""operatingSystemVersion"": ""not available in demo dataset"", ""isMobile"": false, ""mobileDeviceBranding"": ""not available in demo dataset"", ""mobileDeviceModel"": ""not available in demo dataset"", ""mobileInputSelector"": ""not available in demo dataset"", ""mobileDeviceInfo"": ""not available in demo dataset"", ""mobileDeviceMarketingName"": ""not available in demo dataset"", ""flashVersion"": ""not available in demo dataset"", ""language"": ""not available in demo dataset"", ""screenColors"": ""not available in demo dataset"", ""screenResolution"": ""not available in demo dataset"", ""deviceCategory"": ""desktop""}",377306020877927890,"{""continent"": ""Oceania"", ""subContinent"": ""Australasia"", ""country"": ""Australia"", ""region"": ""not available in demo dataset"", ""metro"": ""not available in demo dataset"", ""city"": ""not available in demo dataset"", ""cityId"": ""not available in demo dataset"", ""networkDomain"": ""dodo.net.au"", ""latitude"": ""not available in demo dataset"", ""longitude"": ""not available in demo dataset"", ""networkLocation"": ""not available in demo dataset""}",377306020877927890_1472880147,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"", ""bounces"": ""1"", ""newVisits"": ""1""}","{""campaign"": ""(not set)"", ""source"": ""google"", ""medium"": ""organic"", ""keyword"": ""(not provided)"", ""adwordsClickInfo"": {""criteriaParameters"": ""not available in demo dataset""}}",1472880147,1,1472880147
2,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not available in demo dataset"", ""browserSize"": ""not available in demo dataset"", ""operatingSystem"": ""Windows"", ""operatingSystemVersion"": ""not available in demo dataset"", ""isMobile"": false, ""mobileDeviceBranding"": ""not available in demo dataset"", ""mobileDeviceModel"": ""not available in demo dataset"", ""mobileInputSelector"": ""not available in demo dataset"", ""mobileDeviceInfo"": ""not available in demo dataset"", ""mobileDeviceMarketingName"": "

In [6]:
with open("data/description.txt", mode="r") as fp:
    content = fp.read()
    print(content)

What files do I need?

You will need to download train.csv and test.csv. These contain the data necessary to make predictions for each fullVisitorId listed in sample_submission.csv.

You can also access this data via BigQuery, using the example notebook provided as a way to get started. The data in train.csv is contained in the BigQuery ga_train_set dataset, and the data in test.csv is contained in the ga_test_set dataset, under the kaggle-public-datasets project, accessible through Kernels. In those BigQuery datasets, each day's worth of data is a separate table for more efficient EDA / download.

All information below pertains to the data in both CSV and BigQuery format.
What should I expect the data format to be?

Both train.csv and test.csv contain the columns listed under Data Fields. Each row in the dataset is one visit to the store. Because we are predicting the log of the total revenue per user, be aware that not all rows in test.csv will correspond to a row in the submission, 

### channelGrouping

In [7]:
data.channelGrouping.unique()

array(['Organic Search', 'Referral', 'Paid Search', 'Affiliates',
       'Direct', 'Display', 'Social', '(Other)'], dtype=object)

In [8]:
data.channelGrouping.isnull().sum()

0

In [9]:
data.channelGrouping.isna().sum()

0

In [10]:
channelGroupingMap = {
    "Organic Search": 1,
    "Referral": 2,
    "Paid Search": 3,
    "Affiliates": 4,
    "Direct": 5,
    "Display": 6,
    "Social": 7,
    "(Other)": 8
}

In [11]:
data.channelGrouping = data.channelGrouping.astype("category")

In [12]:
data.channelGrouping.dtypes

CategoricalDtype(categories=['(Other)', 'Affiliates', 'Direct', 'Display',
                  'Organic Search', 'Paid Search', 'Referral', 'Social'],
                 ordered=False)

In [13]:
data.channelGrouping = data.channelGrouping.replace(channelGroupingMap)

In [14]:
data.head(5)

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,1,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not available in demo dataset"", ""browserSize"": ""not available in demo dataset"", ""operatingSystem"": ""Windows"", ""operatingSystemVersion"": ""not available in demo dataset"", ""isMobile"": false, ""mobileDeviceBranding"": ""not available in demo dataset"", ""mobileDeviceModel"": ""not available in demo dataset"", ""mobileInputSelector"": ""not available in demo dataset"", ""mobileDeviceInfo"": ""not available in demo dataset"", ""mobileDeviceMarketingName"": ""not available in demo dataset"", ""flashVersion"": ""not available in demo dataset"", ""language"": ""not available in demo dataset"", ""screenColors"": ""not available in demo dataset"", ""screenResolution"": ""not available in demo dataset"", ""deviceCategory"": ""desktop""}",1131660440785968503,"{""continent"": ""Asia"", ""subContinent"": ""Western Asia"", ""country"": ""Turkey"", ""region"": ""Izmir"", ""metro"": ""(not set)"", ""city"": ""Izmir"", ""cityId"": ""not available in demo dataset"", ""networkDomain"": ""ttnet.com.tr"", ""latitude"": ""not available in demo dataset"", ""longitude"": ""not available in demo dataset"", ""networkLocation"": ""not available in demo dataset""}",1131660440785968503_1472830385,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"", ""bounces"": ""1"", ""newVisits"": ""1""}","{""campaign"": ""(not set)"", ""source"": ""google"", ""medium"": ""organic"", ""keyword"": ""(not provided)"", ""adwordsClickInfo"": {""criteriaParameters"": ""not available in demo dataset""}}",1472830385,1,1472830385
1,1,20160902,"{""browser"": ""Firefox"", ""browserVersion"": ""not available in demo dataset"", ""browserSize"": ""not available in demo dataset"", ""operatingSystem"": ""Macintosh"", ""operatingSystemVersion"": ""not available in demo dataset"", ""isMobile"": false, ""mobileDeviceBranding"": ""not available in demo dataset"", ""mobileDeviceModel"": ""not available in demo dataset"", ""mobileInputSelector"": ""not available in demo dataset"", ""mobileDeviceInfo"": ""not available in demo dataset"", ""mobileDeviceMarketingName"": ""not available in demo dataset"", ""flashVersion"": ""not available in demo dataset"", ""language"": ""not available in demo dataset"", ""screenColors"": ""not available in demo dataset"", ""screenResolution"": ""not available in demo dataset"", ""deviceCategory"": ""desktop""}",377306020877927890,"{""continent"": ""Oceania"", ""subContinent"": ""Australasia"", ""country"": ""Australia"", ""region"": ""not available in demo dataset"", ""metro"": ""not available in demo dataset"", ""city"": ""not available in demo dataset"", ""cityId"": ""not available in demo dataset"", ""networkDomain"": ""dodo.net.au"", ""latitude"": ""not available in demo dataset"", ""longitude"": ""not available in demo dataset"", ""networkLocation"": ""not available in demo dataset""}",377306020877927890_1472880147,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"", ""bounces"": ""1"", ""newVisits"": ""1""}","{""campaign"": ""(not set)"", ""source"": ""google"", ""medium"": ""organic"", ""keyword"": ""(not provided)"", ""adwordsClickInfo"": {""criteriaParameters"": ""not available in demo dataset""}}",1472880147,1,1472880147
2,1,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not available in demo dataset"", ""browserSize"": ""not available in demo dataset"", ""operatingSystem"": ""Windows"", ""operatingSystemVersion"": ""not available in demo dataset"", ""isMobile"": false, ""mobileDeviceBranding"": ""not available in demo dataset"", ""mobileDeviceModel"": ""not available in demo dataset"", ""mobileInputSelector"": ""not available in demo dataset"", ""mobileDeviceInfo"": ""not available in demo dataset"", ""mobileDeviceMarketingName"": ""not available in demo dataset"", ""fla

### socialEngagementType

In [15]:
data.socialEngagementType.unique()

array(['Not Socially Engaged'], dtype=object)

In [16]:
data.socialEngagementType.isnull().sum()

0

In [17]:
data.socialEngagementType.isna().sum()

0

In [18]:
data.socialEngagementType = data.socialEngagementType.astype("category")

In [19]:
data.socialEngagementType = data.socialEngagementType.replace({"Not Socially Engaged": 1})

In [20]:
data.head(5)

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,1,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not available in demo dataset"", ""browserSize"": ""not available in demo dataset"", ""operatingSystem"": ""Windows"", ""operatingSystemVersion"": ""not available in demo dataset"", ""isMobile"": false, ""mobileDeviceBranding"": ""not available in demo dataset"", ""mobileDeviceModel"": ""not available in demo dataset"", ""mobileInputSelector"": ""not available in demo dataset"", ""mobileDeviceInfo"": ""not available in demo dataset"", ""mobileDeviceMarketingName"": ""not available in demo dataset"", ""flashVersion"": ""not available in demo dataset"", ""language"": ""not available in demo dataset"", ""screenColors"": ""not available in demo dataset"", ""screenResolution"": ""not available in demo dataset"", ""deviceCategory"": ""desktop""}",1131660440785968503,"{""continent"": ""Asia"", ""subContinent"": ""Western Asia"", ""country"": ""Turkey"", ""region"": ""Izmir"", ""metro"": ""(not set)"", ""city"": ""Izmir"", ""cityId"": ""not available in demo dataset"", ""networkDomain"": ""ttnet.com.tr"", ""latitude"": ""not available in demo dataset"", ""longitude"": ""not available in demo dataset"", ""networkLocation"": ""not available in demo dataset""}",1131660440785968503_1472830385,1,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"", ""bounces"": ""1"", ""newVisits"": ""1""}","{""campaign"": ""(not set)"", ""source"": ""google"", ""medium"": ""organic"", ""keyword"": ""(not provided)"", ""adwordsClickInfo"": {""criteriaParameters"": ""not available in demo dataset""}}",1472830385,1,1472830385
1,1,20160902,"{""browser"": ""Firefox"", ""browserVersion"": ""not available in demo dataset"", ""browserSize"": ""not available in demo dataset"", ""operatingSystem"": ""Macintosh"", ""operatingSystemVersion"": ""not available in demo dataset"", ""isMobile"": false, ""mobileDeviceBranding"": ""not available in demo dataset"", ""mobileDeviceModel"": ""not available in demo dataset"", ""mobileInputSelector"": ""not available in demo dataset"", ""mobileDeviceInfo"": ""not available in demo dataset"", ""mobileDeviceMarketingName"": ""not available in demo dataset"", ""flashVersion"": ""not available in demo dataset"", ""language"": ""not available in demo dataset"", ""screenColors"": ""not available in demo dataset"", ""screenResolution"": ""not available in demo dataset"", ""deviceCategory"": ""desktop""}",377306020877927890,"{""continent"": ""Oceania"", ""subContinent"": ""Australasia"", ""country"": ""Australia"", ""region"": ""not available in demo dataset"", ""metro"": ""not available in demo dataset"", ""city"": ""not available in demo dataset"", ""cityId"": ""not available in demo dataset"", ""networkDomain"": ""dodo.net.au"", ""latitude"": ""not available in demo dataset"", ""longitude"": ""not available in demo dataset"", ""networkLocation"": ""not available in demo dataset""}",377306020877927890_1472880147,1,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"", ""bounces"": ""1"", ""newVisits"": ""1""}","{""campaign"": ""(not set)"", ""source"": ""google"", ""medium"": ""organic"", ""keyword"": ""(not provided)"", ""adwordsClickInfo"": {""criteriaParameters"": ""not available in demo dataset""}}",1472880147,1,1472880147
2,1,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not available in demo dataset"", ""browserSize"": ""not available in demo dataset"", ""operatingSystem"": ""Windows"", ""operatingSystemVersion"": ""not available in demo dataset"", ""isMobile"": false, ""mobileDeviceBranding"": ""not available in demo dataset"", ""mobileDeviceModel"": ""not available in demo dataset"", ""mobileInputSelector"": ""not available in demo dataset"", ""mobileDeviceInfo"": ""not available in demo dataset"", ""mobileDeviceMarketingName"": ""not available in demo dataset"", ""flashVersion"": ""not available in demo d

### visitId

In [21]:
data.visitId.unique()

array([1472830385, 1472880147, 1472865386, ..., 1483526434, 1483592857,
       1483574474])

In [22]:
data.visitId.isnull().sum()

0

In [23]:
data.visitId.isna().sum()

0

### visitStartTime

In [24]:
data.visitStartTime.unique()

array([1472830385, 1472880147, 1472865386, ..., 1483526434, 1483592864,
       1483574474])

In [25]:
data.visitStartTime.isnull().sum()

0

In [26]:
data.visitStartTime.isna().sum()

0

### visitNumber

In [27]:
data.visitNumber.unique()

array([  1,   2,   3,   5,  11,   4,  57,   6,  56,   7,  20,   8,  15,
         9,  25,  24,  14,  89, 136,  13,  85,  10, 105,  22,  21,  48,
        18,  26,  17,  83,  38,  84,  27,  42, 100,  31,  16,  30,  59,
        50,  19,  51,  29, 160,  52,  12,  63,  23,  47,  49,  28,  34,
       178,  88,  33, 317,  93, 140,  35,  41,  91,  92,  74, 309,  43,
        61,  46,  45,  32, 108,  86, 195,  36, 236,  72, 162, 235,  90,
        76,  77,  94,  96,  73,  78,  97,  87, 304, 106, 107, 389,  68,
        98, 132,  58, 266, 267, 221,  39, 121, 143, 142, 174, 175, 189,
       191,  55, 190, 104, 245, 204,  37,  82, 137, 206, 101,  62, 110,
       156, 158, 157, 159, 138, 109, 194, 193, 147, 254,  71,  53, 144,
       145, 146,  80, 253,  99, 134, 135, 102,  44, 130,  95,  75, 103,
       315,  70,  69, 117, 163, 154, 153, 283,  65, 262, 263, 295, 297,
       298, 219,  64, 296, 155, 141, 169, 170,  60, 207,  54, 187, 133,
       118, 264,  67,  66, 126,  40, 122, 325, 326, 197, 167, 19

In [28]:
data.visitNumber.isnull().sum()

0

In [29]:
data.visitNumber.isna().sum()

0

In [30]:
data.head(5)

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,1,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not available in demo dataset"", ""browserSize"": ""not available in demo dataset"", ""operatingSystem"": ""Windows"", ""operatingSystemVersion"": ""not available in demo dataset"", ""isMobile"": false, ""mobileDeviceBranding"": ""not available in demo dataset"", ""mobileDeviceModel"": ""not available in demo dataset"", ""mobileInputSelector"": ""not available in demo dataset"", ""mobileDeviceInfo"": ""not available in demo dataset"", ""mobileDeviceMarketingName"": ""not available in demo dataset"", ""flashVersion"": ""not available in demo dataset"", ""language"": ""not available in demo dataset"", ""screenColors"": ""not available in demo dataset"", ""screenResolution"": ""not available in demo dataset"", ""deviceCategory"": ""desktop""}",1131660440785968503,"{""continent"": ""Asia"", ""subContinent"": ""Western Asia"", ""country"": ""Turkey"", ""region"": ""Izmir"", ""metro"": ""(not set)"", ""city"": ""Izmir"", ""cityId"": ""not available in demo dataset"", ""networkDomain"": ""ttnet.com.tr"", ""latitude"": ""not available in demo dataset"", ""longitude"": ""not available in demo dataset"", ""networkLocation"": ""not available in demo dataset""}",1131660440785968503_1472830385,1,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"", ""bounces"": ""1"", ""newVisits"": ""1""}","{""campaign"": ""(not set)"", ""source"": ""google"", ""medium"": ""organic"", ""keyword"": ""(not provided)"", ""adwordsClickInfo"": {""criteriaParameters"": ""not available in demo dataset""}}",1472830385,1,1472830385
1,1,20160902,"{""browser"": ""Firefox"", ""browserVersion"": ""not available in demo dataset"", ""browserSize"": ""not available in demo dataset"", ""operatingSystem"": ""Macintosh"", ""operatingSystemVersion"": ""not available in demo dataset"", ""isMobile"": false, ""mobileDeviceBranding"": ""not available in demo dataset"", ""mobileDeviceModel"": ""not available in demo dataset"", ""mobileInputSelector"": ""not available in demo dataset"", ""mobileDeviceInfo"": ""not available in demo dataset"", ""mobileDeviceMarketingName"": ""not available in demo dataset"", ""flashVersion"": ""not available in demo dataset"", ""language"": ""not available in demo dataset"", ""screenColors"": ""not available in demo dataset"", ""screenResolution"": ""not available in demo dataset"", ""deviceCategory"": ""desktop""}",377306020877927890,"{""continent"": ""Oceania"", ""subContinent"": ""Australasia"", ""country"": ""Australia"", ""region"": ""not available in demo dataset"", ""metro"": ""not available in demo dataset"", ""city"": ""not available in demo dataset"", ""cityId"": ""not available in demo dataset"", ""networkDomain"": ""dodo.net.au"", ""latitude"": ""not available in demo dataset"", ""longitude"": ""not available in demo dataset"", ""networkLocation"": ""not available in demo dataset""}",377306020877927890_1472880147,1,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"", ""bounces"": ""1"", ""newVisits"": ""1""}","{""campaign"": ""(not set)"", ""source"": ""google"", ""medium"": ""organic"", ""keyword"": ""(not provided)"", ""adwordsClickInfo"": {""criteriaParameters"": ""not available in demo dataset""}}",1472880147,1,1472880147
2,1,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not available in demo dataset"", ""browserSize"": ""not available in demo dataset"", ""operatingSystem"": ""Windows"", ""operatingSystemVersion"": ""not available in demo dataset"", ""isMobile"": false, ""mobileDeviceBranding"": ""not available in demo dataset"", ""mobileDeviceModel"": ""not available in demo dataset"", ""mobileInputSelector"": ""not available in demo dataset"", ""mobileDeviceInfo"": ""not available in demo dataset"", ""mobileDeviceMarketingName"": ""not available in demo dataset"", ""flashVersion"": ""not available in demo d

### totals

In [31]:
data["transactionRevenue"] = pd.DataFrame(data.totals.apply(json.loads).tolist())[["transactionRevenue"]]
data.head(5)

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime,transactionRevenue
0,1,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not available in demo dataset"", ""browserSize"": ""not available in demo dataset"", ""operatingSystem"": ""Windows"", ""operatingSystemVersion"": ""not available in demo dataset"", ""isMobile"": false, ""mobileDeviceBranding"": ""not available in demo dataset"", ""mobileDeviceModel"": ""not available in demo dataset"", ""mobileInputSelector"": ""not available in demo dataset"", ""mobileDeviceInfo"": ""not available in demo dataset"", ""mobileDeviceMarketingName"": ""not available in demo dataset"", ""flashVersion"": ""not available in demo dataset"", ""language"": ""not available in demo dataset"", ""screenColors"": ""not available in demo dataset"", ""screenResolution"": ""not available in demo dataset"", ""deviceCategory"": ""desktop""}",1131660440785968503,"{""continent"": ""Asia"", ""subContinent"": ""Western Asia"", ""country"": ""Turkey"", ""region"": ""Izmir"", ""metro"": ""(not set)"", ""city"": ""Izmir"", ""cityId"": ""not available in demo dataset"", ""networkDomain"": ""ttnet.com.tr"", ""latitude"": ""not available in demo dataset"", ""longitude"": ""not available in demo dataset"", ""networkLocation"": ""not available in demo dataset""}",1131660440785968503_1472830385,1,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"", ""bounces"": ""1"", ""newVisits"": ""1""}","{""campaign"": ""(not set)"", ""source"": ""google"", ""medium"": ""organic"", ""keyword"": ""(not provided)"", ""adwordsClickInfo"": {""criteriaParameters"": ""not available in demo dataset""}}",1472830385,1,1472830385,NaN
1,1,20160902,"{""browser"": ""Firefox"", ""browserVersion"": ""not available in demo dataset"", ""browserSize"": ""not available in demo dataset"", ""operatingSystem"": ""Macintosh"", ""operatingSystemVersion"": ""not available in demo dataset"", ""isMobile"": false, ""mobileDeviceBranding"": ""not available in demo dataset"", ""mobileDeviceModel"": ""not available in demo dataset"", ""mobileInputSelector"": ""not available in demo dataset"", ""mobileDeviceInfo"": ""not available in demo dataset"", ""mobileDeviceMarketingName"": ""not available in demo dataset"", ""flashVersion"": ""not available in demo dataset"", ""language"": ""not available in demo dataset"", ""screenColors"": ""not available in demo dataset"", ""screenResolution"": ""not available in demo dataset"", ""deviceCategory"": ""desktop""}",377306020877927890,"{""continent"": ""Oceania"", ""subContinent"": ""Australasia"", ""country"": ""Australia"", ""region"": ""not available in demo dataset"", ""metro"": ""not available in demo dataset"", ""city"": ""not available in demo dataset"", ""cityId"": ""not available in demo dataset"", ""networkDomain"": ""dodo.net.au"", ""latitude"": ""not available in demo dataset"", ""longitude"": ""not available in demo dataset"", ""networkLocation"": ""not available in demo dataset""}",377306020877927890_1472880147,1,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"", ""bounces"": ""1"", ""newVisits"": ""1""}","{""campaign"": ""(not set)"", ""source"": ""google"", ""medium"": ""organic"", ""keyword"": ""(not provided)"", ""adwordsClickInfo"": {""criteriaParameters"": ""not available in demo dataset""}}",1472880147,1,1472880147,NaN
2,1,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not available in demo dataset"", ""browserSize"": ""not available in demo dataset"", ""operatingSystem"": ""Windows"", ""operatingSystemVersion"": ""not available in demo dataset"", ""isMobile"": false, ""mobileDeviceBranding"": ""not available in demo dataset"", ""mobileDeviceModel"": ""not available in demo dataset"", ""mobileInputSelector"": ""not available in demo dataset"", ""mobileDeviceInfo"": ""not available in demo dataset"", ""mobileDeviceMarketingName"": ""not available in demo dataset"", ""flashVersion""

In [32]:
data.shape

(903653, 13)

In [33]:
data = data.dropna(subset=["transactionRevenue"])

In [34]:
data.shape

(11515, 13)

In [35]:
data.head(5)

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime,transactionRevenue
752,5,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not available in demo dataset"", ""browserSize"": ""not available in demo dataset"", ""operatingSystem"": ""Linux"", ""operatingSystemVersion"": ""not available in demo dataset"", ""isMobile"": false, ""mobileDeviceBranding"": ""not available in demo dataset"", ""mobileDeviceModel"": ""not available in demo dataset"", ""mobileInputSelector"": ""not available in demo dataset"", ""mobileDeviceInfo"": ""not available in demo dataset"", ""mobileDeviceMarketingName"": ""not available in demo dataset"", ""flashVersion"": ""not available in demo dataset"", ""language"": ""not available in demo dataset"", ""screenColors"": ""not available in demo dataset"", ""screenResolution"": ""not available in demo dataset"", ""deviceCategory"": ""desktop""}",6194193421514403509,"{""continent"": ""Americas"", ""subContinent"": ""Northern America"", ""country"": ""United States"", ""region"": ""Michigan"", ""metro"": ""Detroit MI"", ""city"": ""Ann Arbor"", ""cityId"": ""not available in demo dataset"", ""networkDomain"": ""(not set)"", ""latitude"": ""not available in demo dataset"", ""longitude"": ""not available in demo dataset"", ""networkLocation"": ""not available in demo dataset""}",6194193421514403509_1472843572,1,"{""visits"": ""1"", ""hits"": ""11"", ""pageviews"": ""11"", ""transactionRevenue"": ""37860000"", ""newVisits"": ""1""}","{""campaign"": ""(not set)"", ""source"": ""(direct)"", ""medium"": ""(none)"", ""adwordsClickInfo"": {""criteriaParameters"": ""not available in demo dataset""}, ""isTrueDirect"": true}",1472843572,1,1472843572,37860000
753,1,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not available in demo dataset"", ""browserSize"": ""not available in demo dataset"", ""operatingSystem"": ""Macintosh"", ""operatingSystemVersion"": ""not available in demo dataset"", ""isMobile"": false, ""mobileDeviceBranding"": ""not available in demo dataset"", ""mobileDeviceModel"": ""not available in demo dataset"", ""mobileInputSelector"": ""not available in demo dataset"", ""mobileDeviceInfo"": ""not available in demo dataset"", ""mobileDeviceMarketingName"": ""not available in demo dataset"", ""flashVersion"": ""not available in demo dataset"", ""language"": ""not available in demo dataset"", ""screenColors"": ""not available in demo dataset"", ""screenResolution"": ""not available in demo dataset"", ""deviceCategory"": ""desktop""}",5327166854580374902,"{""continent"": ""Americas"", ""subContinent"": ""Northern America"", ""country"": ""United States"", ""region"": ""New York"", ""metro"": ""New York NY"", ""city"": ""New York"", ""cityId"": ""not available in demo dataset"", ""networkDomain"": ""(not set)"", ""latitude"": ""not available in demo dataset"", ""longitude"": ""not available in demo dataset"", ""networkLocation"": ""not available in demo dataset""}",5327166854580374902_1472844906,1,"{""visits"": ""1"", ""hits"": ""11"", ""pageviews"": ""10"", ""transactionRevenue"": ""306670000""}","{""campaign"": ""(not set)"", ""source"": ""google"", ""medium"": ""organic"", ""keyword"": ""(not provided)"", ""adwordsClickInfo"": {""criteriaParameters"": ""not available in demo dataset""}, ""isTrueDirect"": true}",1472844906,3,1472844906,306670000
799,2,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not available in demo dataset"", ""browserSize"": ""not available in demo dataset"", ""operatingSystem"": ""Linux"", ""operatingSystemVersion"": ""not available in demo dataset"", ""isMobile"": false, ""mobileDeviceBranding"": ""not available in demo dataset"", ""mobileDeviceModel"": ""not available in demo dataset"", ""mobileInputSelector"": ""not available in demo dataset"", ""mobileDeviceInfo"": ""not available in demo dataset"", ""mobileDeviceMarketingName"": ""not available in demo dataset"",

In [36]:
device_json_dump = data.device.apply(json.loads)

In [37]:
type(device_json_dump)

pandas.core.series.Series

In [38]:
device_json_dump.iloc[0]

{'browser': 'Chrome',
 'browserVersion': 'not available in demo dataset',
 'browserSize': 'not available in demo dataset',
 'operatingSystem': 'Linux',
 'operatingSystemVersion': 'not available in demo dataset',
 'isMobile': False,
 'mobileDeviceBranding': 'not available in demo dataset',
 'mobileDeviceModel': 'not available in demo dataset',
 'mobileInputSelector': 'not available in demo dataset',
 'mobileDeviceInfo': 'not available in demo dataset',
 'mobileDeviceMarketingName': 'not available in demo dataset',
 'flashVersion': 'not available in demo dataset',
 'language': 'not available in demo dataset',
 'screenColors': 'not available in demo dataset',
 'screenResolution': 'not available in demo dataset',
 'deviceCategory': 'desktop'}

Based on domain knowledge, this factor will not affect the revenue of the customer.

In [39]:
data.columns

Index(['channelGrouping', 'date', 'device', 'fullVisitorId', 'geoNetwork',
       'sessionId', 'socialEngagementType', 'totals', 'trafficSource',
       'visitId', 'visitNumber', 'visitStartTime', 'transactionRevenue'],
      dtype='object')

### geoNetwork

In [40]:
data.geoNetwork.apply(json.loads).iloc[:3]

752    {'continent': 'Americas', 'subContinent': 'Northern America', 'country': 'United States', 'region': 'Michigan', 'metro': 'Detroit MI', 'city': 'Ann Arbor', 'cityId': 'not available in demo dataset', 'networkDomain': '(not set)', 'latitude': 'not available in demo dataset', 'longitude': 'not available in demo dataset', 'networkLocation': 'not available in demo dataset'}
753    {'continent': 'Americas', 'subContinent': 'Northern America', 'country': 'United States', 'region': 'New York', 'metro': 'New York NY', 'city': 'New York', 'cityId': 'not available in demo dataset', 'networkDomain': '(not set)', 'latitude': 'not available in demo dataset', 'longitude': 'not available in demo dataset', 'networkLocation': 'not available in demo dataset'}
799    {'continent': 'Americas', 'subContinent': 'Northern America', 'country': 'United States', 'region': 'New York', 'metro': 'New York NY', 'city': 'New York', 'cityId': 'not available in demo dataset', 'networkDomain': '(not set)', 'latit

In [42]:
data["country"] = data.geoNetwork.apply(json.loads).apply(lambda x: x["country"])
data["continent"] = data.geoNetwork.apply(json.loads).apply(lambda x: x["continent"])
data["region"] = data.geoNetwork.apply(json.loads).apply(lambda x: x["region"])
data.head(5)

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime,transactionRevenue,country,continent,region
752,5,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not available in demo dataset"", ""browserSize"": ""not available in demo dataset"", ""operatingSystem"": ""Linux"", ""operatingSystemVersion"": ""not available in demo dataset"", ""isMobile"": false, ""mobileDeviceBranding"": ""not available in demo dataset"", ""mobileDeviceModel"": ""not available in demo dataset"", ""mobileInputSelector"": ""not available in demo dataset"", ""mobileDeviceInfo"": ""not available in demo dataset"", ""mobileDeviceMarketingName"": ""not available in demo dataset"", ""flashVersion"": ""not available in demo dataset"", ""language"": ""not available in demo dataset"", ""screenColors"": ""not available in demo dataset"", ""screenResolution"": ""not available in demo dataset"", ""deviceCategory"": ""desktop""}",6194193421514403509,"{""continent"": ""Americas"", ""subContinent"": ""Northern America"", ""country"": ""United States"", ""region"": ""Michigan"", ""metro"": ""Detroit MI"", ""city"": ""Ann Arbor"", ""cityId"": ""not available in demo dataset"", ""networkDomain"": ""(not set)"", ""latitude"": ""not available in demo dataset"", ""longitude"": ""not available in demo dataset"", ""networkLocation"": ""not available in demo dataset""}",6194193421514403509_1472843572,1,"{""visits"": ""1"", ""hits"": ""11"", ""pageviews"": ""11"", ""transactionRevenue"": ""37860000"", ""newVisits"": ""1""}","{""campaign"": ""(not set)"", ""source"": ""(direct)"", ""medium"": ""(none)"", ""adwordsClickInfo"": {""criteriaParameters"": ""not available in demo dataset""}, ""isTrueDirect"": true}",1472843572,1,1472843572,37860000,United States,Americas,Michigan
753,1,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not available in demo dataset"", ""browserSize"": ""not available in demo dataset"", ""operatingSystem"": ""Macintosh"", ""operatingSystemVersion"": ""not available in demo dataset"", ""isMobile"": false, ""mobileDeviceBranding"": ""not available in demo dataset"", ""mobileDeviceModel"": ""not available in demo dataset"", ""mobileInputSelector"": ""not available in demo dataset"", ""mobileDeviceInfo"": ""not available in demo dataset"", ""mobileDeviceMarketingName"": ""not available in demo dataset"", ""flashVersion"": ""not available in demo dataset"", ""language"": ""not available in demo dataset"", ""screenColors"": ""not available in demo dataset"", ""screenResolution"": ""not available in demo dataset"", ""deviceCategory"": ""desktop""}",5327166854580374902,"{""continent"": ""Americas"", ""subContinent"": ""Northern America"", ""country"": ""United States"", ""region"": ""New York"", ""metro"": ""New York NY"", ""city"": ""New York"", ""cityId"": ""not available in demo dataset"", ""networkDomain"": ""(not set)"", ""latitude"": ""not available in demo dataset"", ""longitude"": ""not available in demo dataset"", ""networkLocation"": ""not available in demo dataset""}",5327166854580374902_1472844906,1,"{""visits"": ""1"", ""hits"": ""11"", ""pageviews"": ""10"", ""transactionRevenue"": ""306670000""}","{""campaign"": ""(not set)"", ""source"": ""google"", ""medium"": ""organic"", ""keyword"": ""(not provided)"", ""adwordsClickInfo"": {""criteriaParameters"": ""not available in demo dataset""}, ""isTrueDirect"": true}",1472844906,3,1472844906,306670000,United States,Americas,New York
799,2,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not available in demo dataset"", ""browserSize"": ""not available in demo dataset"", ""operatingSystem"": ""Linux"", ""operatingSystemVersion"": ""not available in demo dataset"", ""isMobile"": false, ""mobileDeviceBranding"": ""not available in demo dataset"", ""mobileDeviceModel"": ""not available in demo dataset"", ""mobileInputSelector"": ""not available in demo dataset"", ""mobileDeviceInfo"": ""not avail

In [43]:
data.country.unique()

array(['United States', 'Puerto Rico', 'Taiwan', 'Switzerland', 'Canada',
       'Mexico', 'Australia', 'Sweden', 'Ireland', 'Israel', 'Germany',
       'United Kingdom', 'New Zealand', 'Argentina', 'Chile', 'Finland',
       'India', 'Venezuela', 'Indonesia', 'St. Lucia', 'Japan', 'China',
       'Colombia', 'Hong Kong', 'Brazil', 'Philippines', '(not set)',
       'Pakistan', 'South Korea', 'Ecuador', 'Singapore', 'Hungary',
       'Greece', 'Russia', 'Kuwait', 'Malaysia', 'Spain', 'Ukraine',
       'Cyprus', 'Romania', 'Nigeria', 'Uruguay', 'Belgium', 'Kenya',
       'Kazakhstan', 'Turkey', 'Saudi Arabia', 'Anguilla', 'Armenia',
       'France', 'United Arab Emirates', 'Thailand', 'Netherlands',
       'South Africa', 'Poland', 'Egypt', 'Italy', 'Guatemala', 'Georgia',
       'El Salvador', 'Lebanon', 'Nicaragua', 'Czechia', 'Denmark',
       'Curaçao', 'Peru', 'Panama', 'Portugal', 'Guadeloupe'],
      dtype=object)

In [46]:
data.continent.unique()

array(['Americas', 'Asia', 'Europe', 'Oceania', '(not set)', 'Africa'],
      dtype=object)

In [47]:
data.region.unique()

array(['Michigan', 'New York', 'California',
       'not available in demo dataset', 'Washington', 'Illinois',
       'Oregon', 'District of Columbia', 'Massachusetts', '(not set)',
       'Virginia', 'Zurich', 'Georgia', 'Texas', 'Ontario',
       'Pennsylvania', 'Nevada', 'North Carolina',
       'State of Rio de Janeiro', 'Victoria', 'County Dublin',
       'Tel Aviv District', 'Taipei City', 'Colorado', 'New Jersey',
       'Mexico City', 'Florida', 'New Taipei City', 'England',
       'Santiago Metropolitan Region', 'Nebraska', 'South Carolina',
       'Utah', 'Tennessee', 'Zulia', 'Alberta', 'Jakarta',
       'Stockholm County', 'Dublin City', 'Arizona', 'Ohio', 'Seoul',
       'Quebec', 'Attica', 'Iowa', 'British Columbia', 'Gujarat',
       'Catalonia', 'New South Wales',
       'Federal Territory of Kuala Lumpur', 'Missouri', 'Vienna',
       'Bucharest', 'Tokyo', 'State of Sao Paulo', 'Kanagawa Prefecture',
       'Maryland', 'Minnesota', 'Delhi', 'Istanbul', 'Indiana',
     

In [45]:
for x in ["country", "region", "continent"]:
    print("isnull:", data[[x]].isnull().sum())
    print("isna:", data[[x]].isna().sum())

isnull: country    0
dtype: int64
isna: country    0
dtype: int64
isnull: region    0
dtype: int64
isna: region    0
dtype: int64
isnull: continent    0
dtype: int64
isna: continent    0
dtype: int64
